In [3]:
import os
from pyspark.sql import SparkSession

In [6]:
spark = (
        SparkSession.builder
        .appName("NYTaxyDataAnalysis")
        .master("spark://spark-master:7077")
        .config("spark.driver.host", "jupyterlab")
        .config("spark.ui.port", "4040")
        .config("spark.driver.memory", "2g")
        .config("spark.executor.cores", "1")
        .config("spark.executor.memory", "4g")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") 
        .config("spark.sql.shuffle.partitions", "8")
        .getOrCreate()
    )

In [ ]:
BUCKET_NAME = "nyc-taxi-data" 
RAW_DATA_PATH = f"s3a://{BUCKET_NAME}/taxi_zones/taxi_zone_lookup.csv"

In [8]:
df = spark.read.csv(RAW_DATA_PATH, header=True, inferSchema=True)

25/06/24 13:51:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [9]:
df.show(5, truncate=False)

+----------+-------------+-----------------------+------------+
|LocationID|Borough      |Zone                   |service_zone|
+----------+-------------+-----------------------+------------+
|1         |EWR          |Newark Airport         |EWR         |
|2         |Queens       |Jamaica Bay            |Boro Zone   |
|3         |Bronx        |Allerton/Pelham Gardens|Boro Zone   |
|4         |Manhattan    |Alphabet City          |Yellow Zone |
|5         |Staten Island|Arden Heights          |Boro Zone   |
+----------+-------------+-----------------------+------------+
only showing top 5 rows



In [10]:
pg_user = os.getenv("POSTGRES_USER")
pg_password = os.getenv("POSTGRES_PASSWORD")
pg_db = os.getenv("POSTGRES_DB")
postgres_url = f"jdbc:postgresql://postgres-gis:5432/{pg_db}"
    
table_name = "nyctaxi.taxi_zone_lookup" 
    
connection_properties = {
    "user": pg_user,
    "password": pg_password,
    "driver": "org.postgresql.Driver"
}

In [11]:
(df.write
    .mode("overwrite")
    .jdbc(
                url=postgres_url,
                table=table_name,
                properties=connection_properties
         )
)